In [1]:
%reset -f
%pylab inline
pylab.rcParams['figure.figsize'] = (12,10) # setting default figure size
pylab.style.use('ggplot') # setting style

Populating the interactive namespace from numpy and matplotlib


In [2]:
import pickle
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from pandas.tseries.offsets import *
from datetime import datetime as dt
from pandas_datareader import data, famafrench
import wrds

db = wrds.Connection(wrds_username='fannywu')

S:\Users\sheha\anaconda3\envs\sentinel_prime\lib\site-packages\pandas_datareader\compat\__init__.py:7: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  from pandas.util.testing import assert_frame_equal


Enter your WRDS username [sheha]:fannywu
Enter your password:········
WRDS recommends setting up a .pgpass file.
You can find more info here:
https://www.postgresql.org/docs/9.5/static/libpq-pgpass.html.
There was an error with your password.


OperationalError: (psycopg2.OperationalError) FATAL:  PAM authentication failed for user "fannywu"

(Background on this error at: http://sqlalche.me/e/e3q8)

In [ ]:
len(permno_list)

In [ ]:
permno_list = pd.read_csv('permnolist.csv')['permno'].tolist()
ff_data = data.DataReader('F-F_Research_Data_Factors_daily', 'famafrench', start='1950-10-01')[0][['Mkt-RF','RF']]/100
ff_data.rename(columns= {'Mkt-RF':'mkt_rf','RF':'rf'}, inplace=True)
ff_data.index =  pd.to_datetime(ff_data.index.astype(str))

In [4]:
vol_lkbk = 252
minvol_lkbk = 120
corr_lkbk = 252*5
mincorr_lkbk = 252*2
ff_data['mkt_rf_vol'] = ff_data['mkt_rf'].rolling(vol_lkbk, min_periods=minvol_lkbk).std()

In [6]:
beta_dict = {}
misc_dict = {}
N = len(permno_list)

for n, permno in enumerate(permno_list):
    
    stock_query =  """
                  select a.permno, a.permco, a.date, b.shrcd, b.exchcd, a.ret
                  from crsp.dsf as a
                  left join crsp.dsenames as b
                  on a.permno=b.permno
                  and b.namedt<=a.date
                  and a.date<=b.nameendt
                  where a.date between '01/01/1920' and '12/31/2020'
                  and b.exchcd between 1 and 3
                  and a.permno = {} 
                  """.format(permno)
    stock_df = db.raw_sql(stock_query)
    
    stock_df.index = pd.to_datetime(stock_df['date'])
    stock_df.drop(columns='date', inplace=True)
    
    if len(stock_df.dropna()) < mincorr_lkbk: continue
        
    df = pd.concat([stock_df, ff_data.dropna()], axis=1).dropna()
    df['rx'] = df['ret'] - df['rf']
    df['rx_vol'] = df['rx'].rolling(vol_lkbk, min_periods=minvol_lkbk).std()
    df['rho'] = df[['rx','mkt_rf']].rolling(corr_lkbk, min_periods=mincorr_lkbk).corr(df['rx'])['mkt_rf']
    df['beta'] = df['rho'].multiply(df['rx_vol'].divide(df['mkt_rf_vol'], axis=0), axis=0)
    
    beta_dict[permno] = df['beta'].dropna()
    misc_dict[permno] = df[['mkt_rf_vol', 'rx_vol', 'rho']].dropna()
    
    if n % 500 is 0:
        print('{:.2f}% of permnos completed...'.format(n*100/N))

monthly_beta_dict = {}
for permno in list(beta_dict.keys()):
    monthly_beta_dict[permno] = beta_dict[permno].resample('M').last()

0.00% of permnos completed...
1.71% of permnos completed...
3.43% of permnos completed...
5.14% of permnos completed...
6.85% of permnos completed...
8.57% of permnos completed...
10.28% of permnos completed...
11.99% of permnos completed...
13.70% of permnos completed...
15.42% of permnos completed...
17.13% of permnos completed...
18.84% of permnos completed...
20.56% of permnos completed...
22.27% of permnos completed...
23.98% of permnos completed...
25.70% of permnos completed...
27.41% of permnos completed...
29.12% of permnos completed...
30.83% of permnos completed...
32.55% of permnos completed...
35.97% of permnos completed...
37.69% of permnos completed...
39.40% of permnos completed...
42.83% of permnos completed...
44.54% of permnos completed...
46.25% of permnos completed...
47.96% of permnos completed...
49.68% of permnos completed...
51.39% of permnos completed...
53.10% of permnos completed...
54.82% of permnos completed...
56.53% of permnos completed...
58.24% of perm

In [33]:
beta_df

,permno,datadate_mend,beta
0,25881,1972-11-30,1.932727
1,25881,1972-12-31,1.857990
2,25881,1973-01-31,1.681233
3,25881,1973-02-28,1.584574
4,25881,1973-03-31,1.420964
...,...,...,...
3327188,17884,2020-11-30,0.378153
3327189,17884,2020-12-31,0.384550
3327190,18313,2020-12-31,0.775187
3327191,18144,2020-11-30,1.034943


In [12]:
with open('beta_dict.pkl', 'wb') as handle:
    pickle.dump(beta_dict, handle, protocol=pickle.HIGHEST_PROTOCOL)
    
with open('misc_dict.pkl', 'wb') as handle:
    pickle.dump(misc_dict, handle, protocol=pickle.HIGHEST_PROTOCOL)
    
with open('monthly_beta_dict.pkl', 'wb') as handle:
    pickle.dump(monthly_beta_dict, handle, protocol=pickle.HIGHEST_PROTOCOL)